# Lets build GPT by Andrej

In [1]:
import tiktoken

In [2]:
enc = tiktoken.get_encoding('gpt2')

In [3]:
enc.n_vocab

50257

In [4]:
enc.encode("hii h ii hi there")

[71, 4178, 289, 21065, 23105, 612]

In [5]:
print('\n\n'.join([f'{i:<7}:{enc.decode([i])}' for i in [71, 4178, 289, 21065, 23105, 612]]))

71     :h

4178   :ii

289    : h

21065  : ii

23105  : hi

612    : there


In [6]:
filename = 'dataset/esenin-royallib.ru.txt'

In [7]:
with open(filename, 'r', encoding='utf-8') as f:
    text = f.read()

In [8]:
print(f'txt file length: {len(text)}')

txt file length: 249624


In [9]:
print(text[:1000])

Вот уж вечер. Роса
Блестит на крапиве.
Я стою у дороги,
Прислонившись к иве.

От луны свет большой
Прямо на нашу крышу.
Где-то песнь соловья
Вдалеке я слышу.

Хорошо и тепло,
Как зимой у печки.
И березы стоят,
Как большие свечки.

И вдали за рекой,
Видно, за опушкой,
Сонный сторож стучит
Мертвой колотушкой.



Там, где капустные грядки
Красной водой поливает восход,
Клененочек маленький матке
Зеленое вымя сосет.

Поет зима — аукает,
Мохнатый лес баюкает
Стозвоном сосняка.
Кругом с тоской глубокою
Плывут в страну далекую
Седые облака.

А по двору метелица
Ковром шелковым стелется,
Но больно холодна.
Воробышки игривые,
Как детки сиротливые,
Прижались у окна.

Озябли пташки малые,
Голодные, усталые,
И жмутся поплотней.
А вьюга с ревом бешеным
Стучит по ставням свешенным
И злится все сильней.

И дремлют пташки нежные
Под эти вихри снежные
У мерзлого окна.
И снится им прекрасная,
В улыбках солнца ясная
Красавица весна.



Ты поила коня из горстей в поводу,
Отражаясь, березы ломались в пруду

In [10]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !#&()*,-.0123456789:;<>?ims «»АБВГДЕЖЗИКЛМНОПРСТУФХЦЧШЩЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё—…
95


In [11]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l : ''.join([itos[i] for i in l])

print(encode("Привет тут"))
print(decode(encode("Привет тут")))

[46, 76, 68, 62, 65, 78, 1, 78, 79, 78]
Привет тут


In [12]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([249624]) torch.int64
tensor([34, 74, 78,  1, 79, 66,  1, 62, 65, 83, 65, 76, 10,  1, 47, 74, 77, 60,
         0, 33, 71, 65, 77, 78, 68, 78,  1, 73, 60,  1, 70, 76, 60, 75, 68, 62,
        65, 10,  0, 59,  1, 77, 78, 74, 90,  1, 79,  1, 64, 74, 76, 74, 63, 68,
         8,  0, 46, 76, 68, 77, 71, 74, 73, 68, 62, 84, 68, 77, 88,  1, 70,  1,
        68, 62, 65, 10,  0,  0, 45, 78,  1, 71, 79, 73, 87,  1, 77, 62, 65, 78,
         1, 61, 74, 71, 88, 84, 74, 69,  0, 46, 76, 91, 72, 74,  1, 73, 60,  1,
        73, 60, 84, 79,  1, 70, 76, 87, 84, 79, 10,  0, 35, 64, 65,  9, 78, 74,
         1, 75, 65, 77, 73, 88,  1, 77, 74, 71, 74, 62, 88, 91,  0, 34, 64, 60,
        71, 65, 70, 65,  1, 91,  1, 77, 71, 87, 84, 79, 10,  0,  0, 52, 74, 76,
        74, 84, 74,  1, 68,  1, 78, 65, 75, 71, 74,  8,  0, 41, 60, 70,  1, 67,
        68, 72, 74, 69,  1, 79,  1, 75, 65, 83, 70, 68, 10,  0, 40,  1, 61, 65,
        76, 65, 67, 87,  1, 77, 78, 74, 91, 78,  8,  0, 41, 60, 70,  1, 61, 74,
       

In [13]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [14]:
block_size = 8            # context length
train_data[:block_size+1]

tensor([34, 74, 78,  1, 79, 66,  1, 62, 65])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target: {target}')

when input is tensor([34]) the target: 74
when input is tensor([34, 74]) the target: 78
when input is tensor([34, 74, 78]) the target: 1
when input is tensor([34, 74, 78,  1]) the target: 79
when input is tensor([34, 74, 78,  1, 79]) the target: 66
when input is tensor([34, 74, 78,  1, 79, 66]) the target: 1
when input is tensor([34, 74, 78,  1, 79, 66,  1]) the target: 62
when input is tensor([34, 74, 78,  1, 79, 66,  1, 62]) the target: 65


In [16]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target: {target}')
        

inputs:
torch.Size([4, 8])
tensor([[60, 71,  8,  0, 44, 65,  1, 78],
        [ 0, 44, 65, 79, 66, 65, 71, 88],
        [73, 88,  1, 68,  1, 67, 62, 65],
        [60, 78, 74, 90,  1, 75, 65, 83]])
targets:
torch.Size([4, 8])
tensor([[71,  8,  0, 44, 65,  1, 78, 74],
        [44, 65, 79, 66, 65, 71, 88,  1],
        [88,  1, 68,  1, 67, 62, 65, 73],
        [78, 74, 90,  1, 75, 65, 83, 60]])
----
when input is [60] the target: 71
when input is [60, 71] the target: 8
when input is [60, 71, 8] the target: 0
when input is [60, 71, 8, 0] the target: 44
when input is [60, 71, 8, 0, 44] the target: 65
when input is [60, 71, 8, 0, 44, 65] the target: 1
when input is [60, 71, 8, 0, 44, 65, 1] the target: 78
when input is [60, 71, 8, 0, 44, 65, 1, 78] the target: 74
when input is [0] the target: 44
when input is [0, 44] the target: 65
when input is [0, 44, 65] the target: 79
when input is [0, 44, 65, 79] the target: 66
when input is [0, 44, 65, 79, 66] the target: 65
when input is [0, 44, 65, 79,

In [17]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)  # B x 1
            idx = torch.cat((idx, idx_next), dim=1)  #  B x T+1
        return idx
        
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(loss.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([])
tensor(5.1501, grad_fn=<NllLossBackward0>)

»Ю;6;«Я«.29еm 1ф4эайн-б!?ЯЭЖЗ#иН&Кпс; У!кЯУ:рёыых;М:СПм)НхОятБ,жв6джЖе(фьффт#К*
8ияр2оя1Шн?&кБД4mНЖ7


In [18]:
#expected loss (analytically)
-torch.log(torch.tensor([1/vocab_size]))

tensor([4.5539])

In [19]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-4)

In [20]:
batch_size = 32
for steps in range(100000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


2.614084482192993


In [21]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))



Пре:
Пра я ващист устодилась енег я о ре
Наштовастют
Скрю
Нед х,
Узероренот
И жьстотемеру,
Чтроны, говый пометубожиниястешатьк пе Госк лоя рнаки прыхо в масть веслолоды лийче,
И ль.
Вья, от претило пая босто нон уй сошалиедясонуй яц! выбость м всмуй гопороте ст снишаю,
Проливенают, ром,
Недазмадрел


In [23]:
# exploring averaging out the previous by t (beeeee)

In [25]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [26]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b,t] = torch.mean(xprev,0)

In [28]:
xbow.shape

torch.Size([4, 8, 2])